This project is to explore, segment, and cluster the neighborhoods in the city of Toronto

## I have used all three question in the single notebook


In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [12]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [13]:
soup = BeautifulSoup(source, 'xml')

AttributeError: 'NoneType' object has no attribute 'find_all'